# Containers

Here we look at options for performing operations under Docker containers.

## Restart container (`restart`)

You can use the `docker container restart <container>` command, or shorter - `docker restart container`, to restart a given Docker container. 

---

We'll look at this command on the image, which prints `hello world` to the logs every time it starts, and locks itself into an infinite cycle to keep running in deatched mode.

The following cell creates such an image and runs a container based on it.

In [5]:
cat << EOF > dockerfile
FROM alpine
CMD echo "hello world" && while true; do :; done
EOF

docker build -t restart_example . &> /dev/null
docker run -itd --rm --name restart_example restart_example

1dd886b39ea60b5c7d8e884a0267cd2da134327ea2ace7f8f85db560d9520f5f


With `docker logs restart_example` we can get what was printed to the standart stream of the container.

In [6]:
docker logs restart_example

hello world


As expected, this is just a `hello world` message.

Now run `docker restart` on the container in question.

In [7]:
docker restart restart_example

restart_example


And check logs again.

In [8]:
docker logs restart_example

hello world
hello world


There are now two `hello world` messages, indicating that the container has been restarted.

Finally - clean the environment of sample containers/images/files.

In [9]:
docker stop restart_example &> /dev/null
docker rmi restart_example &> /dev/null
rm dockerfile

## Activate stopped (`start`)

Using command `docker start` you can return to execution stopped contianer.

---

The following example creates container and stops it. Displayed the output of the `docker ps -a` command and there is `Exited` at the status field.

In [16]:
docker run --name test_container -td alpine &> /dev/null
docker stop test_container &> /dev/null
docker ps -a

CONTAINER ID   IMAGE     COMMAND     CREATED          STATUS                                PORTS     NAMES
1bc3217411a6   alpine    "/bin/sh"   10 seconds ago   Exited (137) Less than a second ago             test_container


So afte executing `start` command we got our container back.

In [15]:
docker start test_container
docker ps

docker stop test_container
docker mr test_container

/ # 
CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


## Execute in (`exec`)

`exec` command allows you to execute something in the container.

For more details check [specific page](containers/execute_in.ipynb).

---

So in the following example for the alpine container, we first install `fortune` utility and then run it.

In [3]:
docker run --name exec_example --rm -idt alpine &> /dev/null

docker exec exec_example apk add fortune &> /dev/null
docker exec exec_example fortune

docker stop exec_example &> /dev/null

Once Law was sitting on the bench
	And Mercy knelt a-weeping.
"Clear out!" he cried, "disordered wench!
	Nor come before me creeping.
Upon your knees if you appear,
'Tis plain you have no standing here."

Then Justice came.  His Honor cried:
	"YOUR states? -- Devil seize you!"
"Amica curiae," she replied --
	"Friend of the court, so please you."
"Begone!" he shouted -- "There's the door --
I never saw your face before!"
		-- Ambrose Bierce, "The Devil's Dictionary"


The `-it` option is extremely useful as it enables pseudo-TTY and allows you to use containers in interactive mode. In simpler terms, it lets you access the shell of the container directly.

Due to the limitations of the Jupyter notebook, we can't demonstrate this directly here, but you can try it out in your terminal with the following commands:

```bash
# This command creates and runs a container named 'test_container' using the Alpine image
docker run -itd --name test_container alpine

# This command opens an interactive shell in the 'test_container'
docker exec -it test_container sh
```

## Copy files (`cp`)

You can copy files from containers to host and viceversy by usind syntax `docker cp <from> <to>`.

---

The following cell starts two docker containers, which we'll use for the following examples. Note that the second container has a `temp_file' with a `secret_message`.

In [7]:
docker run -itd --name container1 --rm alpine
docker run -itd \
    --name container2 --rm alpine sh \
    -c "echo secret_message > temp_file && tail -f /dev/null"

4f7d03227bc74de0d2c0538cef98869bb263d9cca3bd1661bbdd8c59278f7648
aa4da98572298971e7142dda1858909c0288b2830541f752808dbb128293e353


To copy from the container to the host, you must use the syntax `docker cp <container name>:<container path> <host path>`.

The following cell demonstrates copying `temp_file` from `container2` to the host. After this `cat other_file` form host will print `secret_message`.

In [9]:
docker cp container2:temp_file temp_file
cat temp_file

Successfully copied 2.05kB to /home/f.kobak@maxbit.local/Documents/knowledge/Docker/temp_file
secret_message


To copy from the host to the container, you must use the syntax `docker cp <host path> <container name>:<container path>`.

The following cell shows how to copy `temp_file` to `container1`. Then `cat other_file` from the container and print `secret_message`.

In [16]:
docker cp temp_file container1:temp_file
docker exec container1 cat temp_file

Successfully copied 3.07kB to container1:temp_file
secret_message


**Note** Docker doesn't support copying from container to container. The following cell shows what happens when you try.

In [17]:
docker cp container1:temp_file container2:temp_file2

copying between containers is not supported


: 1

Cleaning up the environment after our experiments.

In [18]:
docker stop container1 container2
rm temp_file

container1
container2


## Check logs (`logs`)

The Docker `logs` command allows you to view the standard output of the Docker container. For more details check [this](containers/check_logs.ipynb).

---

The following example runs a docker container which occasionally prints the current time to the container's standard output. And then uses `docker logs` to show the output of this container.

In [56]:
docker run \
    --name test_container -d --rm alpine \
    sh -c "while true; do $(echo date "+%T"); sleep 1; done" &> /dev/null
sleep 5

docker logs test_container
docker stop test_container &> /dev/null

14:09:07
14:09:08
14:09:09
14:09:10
14:09:11
14:09:12


## Monitoring (`stats`)

The `docker stats` command provides real-time monitoring of resource usage by Docker containers. By default, it streams output to the terminal, continuously refreshing the statistics. To obtain a single snapshot of the stats, use the `--no-stream` option, which outputs the results only once. 

---

The following cell demonstrates how to use the `docker stats` command to check the state of your container. 

In [5]:
docker run -itd --rm --name stats_test alpine &> /dev/null 

docker stats --no-stream

docker stop stats_test &> /dev/null

CONTAINER ID   NAME         CPU %     MEM USAGE / LIMIT   MEM %     NET I/O     BLOCK I/O   PIDS
ee606c8d43d4   stats_test   0.00%     508KiB / 31.01GiB   0.00%     526B / 0B   0B / 0B     1
